In [7]:
import pandas as pd 
import numpy as np

In [8]:
df = pd.read_csv('Walmart.csv')
print(df.head())

   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


In [9]:
target_col = 'Weekly_Sales'
feature_col = ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
# может перевести номер филиала в категориальный признак ?

Проверим на выбросы 
Sales - ok
CPI - ok
Temperature - idk. Is 100 tem ok ?
Fuel_Price - ok
Unemployment - ok 

In [10]:
traget_blowout_col = 'Unemployment'
sorted_df = df.sort_values(by= traget_blowout_col)
#print(sorted_df[traget_blowout_col].head(15))
#print(sorted_df[traget_blowout_col].tail(15))

In [11]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\Bogdan\AppData\Local\Temp\ipykernel_6356\3784545759.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
Store,1.000000,-0.335332,-0.000000,-0.022659,0.060023,-0.209492,0.223531
Weekly_Sales,-0.335332,1.000000,0.036891,-0.063810,0.009464,-0.072634,-0.106176
Holiday_Flag,-0.000000,0.036891,1.000000,-0.155091,-0.078347,-0.002162,0.010960
Temperature,-0.022659,-0.063810,-0.155091,1.000000,0.144982,0.176888,0.101158
Fuel_Price,0.060023,0.009464,-0.078347,0.144982,1.000000,-0.170642,-0.034684
CPI,-0.209492,-0.072634,-0.002162,0.176888,-0.170642,1.000000,-0.302020
Unemployment,0.223531,-0.106176,0.010960,0.101158,-0.034684,-0.302020,1.000000


# divide on train and validation

In [12]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
X = df[feature_col]
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Начну со своей реализации

In [15]:
#L1 reg
class MyLinearRegression(object):
    def __init__(self, alpha=0.01, epochs=1000, l1_penalty=0.01):
        self.alpha = alpha
        self.epochs = epochs
        self.l1_penalty = l1_penalty
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            # Вычисляем предсказания модели
            y_pred = X.dot(self.weights) + self.bias

            # Вычисляем ошибку
            error = y - y_pred

            # Обновляем веса и смещение (bias) с учетом L1 регуляризации
            d_w = (-2/n_samples) * X.T.dot(error) + self.l1_penalty * np.sign(self.weights)
            d_b = (-2/n_samples) * np.sum(error)
            self.weights -= self.alpha * d_w
            self.bias -= self.alpha * d_b

    def predict(self, X):
        return X.dot(self.weights) + self.bias
        # (например, в fit), используйте self.w

    def loss(self, y, p):
        return np.mean(np.power(y-p, 2))

In [17]:
model1 = MyLinearRegression()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Расчет R^2
r_squared = r2_score(y_test, y_pred)
print("R^2:", r_squared)

Mean Squared Error (MSE): 298060085707.63605
R^2: 0.13298635788663604


# Sklearn Linear Regression

In [18]:
models = [LinearRegression(), Lasso(), Ridge(), ElasticNet()]

for model in models:
    # Обучение модели на обучающих данных
    model.fit(X_train, y_train)

    # Предсказание на тестовых данных
    y_pred = model.predict(X_test)
    
    print(model.__class__.__name__)
    mse = mean_squared_error(y_test, y_pred)
    print("\tMSE:", mse)

    # Расчет R^2
    r_squared = r2_score(y_test, y_pred)
    print("\tR^2:", r_squared)

LinearRegression
	MSE: 298060113941.9431
	R^2: 0.13298627575712385
Lasso
	MSE: 298060073231.4275
	R^2: 0.13298639417812053
Ridge
	MSE: 298060600864.94525
	R^2: 0.13298485936859783
ElasticNet
	MSE: 304468058555.57495
	R^2: 0.1143464924908223
